<a href="https://colab.research.google.com/github/yesayayova/smoking-detection/blob/main/Smoking_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Smoking Detection

This project contain program to detect the peoples are smoking in the room. The stage of this program as folows:

<ul>
  <li>Detecting person with YOLOv4 Darknet by <a href"https://github.com/AlexeyAB/darknet">AlexeyAB</a></li>
  <li>Take the detected person and classify using trained model</li>
  <li>If there are a smoking person, then captures the frame for the documentation</li>  

In [1]:
import cv2
import numpy as np

## Download dan Install YOLOv4 Darknet

In [ ]:
#clone yolov4
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
#setting makefile
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!sed -i 's/LIBSO=0/LIBSO=1/' Makefile

In [ ]:
#activiting makefile
!make

In [ ]:
# Download yolov4-csp weight
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1V3vsIaxAlGWvK4Aar9bAiK5U0QFttKwq" -O yolov4-csp.weights && rm -rf /tmp/cookies.txt

## Object Detection with YOLOv4

In [ ]:
# import darknet functions to perform object detections
from darknet import *
# load in our YOLOv4 architecture network
network, class_names, class_colors = load_network("cfg/yolov4-csp.cfg", "cfg/coco.data", "yolov4-csp.weights")
width = network_width(network)
height = network_height(network)

# darknet helper function to run detection on image
def darknet_helper(img, width, height):
  darknet_image = make_image(width, height, 3)
  img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  img_resized = cv2.resize(img_rgb, (width, height),
                              interpolation=cv2.INTER_LINEAR)

  # get image ratios to convert bounding boxes to proper size
  img_height, img_width, _ = img.shape
  width_ratio = img_width/width
  height_ratio = img_height/height

  # run model on darknet style image to get detections
  copy_image_from_bytes(darknet_image, img_resized.tobytes())
  detections = detect_image(network, class_names, darknet_image)
  free_image(darknet_image)
  return detections, width_ratio, height_ratio

## Read and save the video result

In [ ]:
from tensorflow.keras.models import load_model

cap = cv2.VideoCapture("/content/Drive/MyDrive/example.mp4")
model = load_model("/content/drive/MyDrive/model_inceptionv3.h5")

frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

out = cv2.VideoWriter('/content/drive/MyDrive/niko-smoking-2.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame_width,frame_height))

# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

id_frame = 0

# Read until video is completed
while (cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:

    # Display the resulting frame
    detections, width_ratio, height_ratio = darknet_helper(frame, width, height)

    for label, confidence, bbox in detections:
      if label=='person':
        left, top, right, bottom = bbox2points(bbox)
        left, top, right, bottom = int(left * width_ratio), int(top * height_ratio), int(right * width_ratio), int(bottom * height_ratio)
        image_croped = frame[top:bottom, left:right]
        classes= [0]

        try:
          image_croped = cv2.resize(image_croped,(150,150))
          image_croped = image_croped/255.0
          x = np.expand_dims(image_croped, axis=0)

          images = np.vstack([x])
          classes = model.predict(images)
        except:
          continue

        if classes[0] > 0.7:
          cv2.rectangle(frame, (left, top), (right, bottom), (0,0,255), 2)
          cv2.putText(frame, "{} [{:.2f}]".format("smoking", float(classes[0])),
                            (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (0, 0, 255), 2)
          
          #buat nyimpen foto kalo kedetect setiap frame kelipatan 50
          if id_frame % 50 == 0 :
            cv2.imread("content/drive/MyDrive/dataset/detect{}".format(id_frame))

        else:
          cv2.rectangle(frame, (left, top), (right, bottom), (0,255,0), 2)
          cv2.putText(frame, "{} [{:.2f}]".format("smoking", float(classes[0])),
                            (left, top - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5,
                            (0, 255, 0), 2)
          
    out.write(frame)
    print("writed frame {}".format(id_frame))
    id_frame += 1
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break

  # Break the loop
  else: 
    break

# When everything done, release the video capture object
cap.release()
out.release()
# Closes all the frames
cv2.destroyAllWindows()